# BP Prediction and ABP Estimation End-to-End Pipelines

# Test GPU (Optional)
Before Starting, kindly check the available GPU from the Google Server, GPU model and other related information. It might help!

In [1]:
import torch
print("Is CUDA enabled GPU Available?", torch.cuda.is_available())
print("GPU Number:", torch.cuda.device_count())
print("Current GPU Index:", torch.cuda.current_device())
print("GPU Type:", torch.cuda.get_device_name(device=None))
print("GPU Capability:", torch.cuda.get_device_capability(device=None))
print("Is GPU Initialized yet?", torch.cuda.is_initialized())

Is CUDA enabled GPU Available? False
GPU Number: 0


AssertionError: Torch not compiled with CUDA enabled

# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/GDrive')

Move to the Target Directory

In [ ]:
%cd "/content/GDrive/MyDrive/Colab_Notebooks/Research/PPG2ABP"

List the Files and Folders Located in the Current Directory

In [ ]:
!ls

## Evaluation of Predicting ABP Waveforms

Here, we present an interactive CLI to predict the ABP waveform from PPG signal from the test data. Ground truth, prediction from approximation network and refinement network are presented, and a comparison is also demonstrated


#Import Libraries

In [2]:
import os
import h5py
import scipy
import random
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io as sio
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from pathlib import Path
from tqdm import tqdm

In [3]:
from ml_models import *
from Evaluation_Metrics_ABP import *
from UNet_1DCNN import UNet
from BCDUNet_1DCNN import BCDUNet
from SEDUNet_1DCNN import SEDUNet
from Dense_Inception_UNet_1DCNN import Dense_Inception_UNet

In [4]:
%matplotlib inline
sns.set_style('white')

# Set Global Constants

In [5]:
num_channel = 4
fold_num = 1

# Import and Prepare Data

Note: Data pre-processing was mainly conducted in MATLAB. Here, data was loaded from the GDrive and prepared for Deep Learning

### Import Dataset for Train and Test on UCI

In [6]:
fl_Train_1 = h5py.File(os.path.join('UCI_Dataset_Part_1_Preprocessed (1).h5'), 'r')
fl_Train_2 = h5py.File(os.path.join('UCI_Dataset_Part_2_Preprocessed (1).h5'), 'r')
fl_Train_3 = h5py.File(os.path.join('UCI_Dataset_Part_3_Preprocessed.h5'), 'r')
print(fl_Train_1['PPG'].shape)
print(fl_Train_2['PPG'].shape)
print(fl_Train_3['PPG'].shape)

(47441, 1024)
(53001, 1024)
(40110, 1024)


In [7]:
fl_Test = h5py.File(os.path.join('UCI_Dataset_Part_4_Preprocessed.h5'), 'r')  # load UCI Test Data
fl_Test['PPG'].shape

(50646, 1024)

### Prepare Train Data


In [8]:
# intialize train data
length = 1024
X_Train_All = []
Y_Train_BP_All = []
Y_Train_ABP_All = []
PPG_All = []
APG_All = []
VPG_All = []
ECG_All = []

if num_channel == 1:

    X_Train = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_1['PPG'].shape[0]), desc='Preparing Train Data Part 1'):
        X_Train.append(np.array(fl_Train_1['PPG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_1['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_1['ABP_RNorm'][i][:length]).reshape(length, 1))
    
    X_Train = np.array(X_Train)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    X_Train_All = X_Train
    Y_Train_BP_All = Y_Train_BP
    Y_Train_ABP_All = Y_Train_ABP

    X_Train = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_2['PPG'].shape[0]), desc='Preparing Train Data Part 2'):
        X_Train.append(np.array(fl_Train_2['PPG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_2['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_2['ABP_RNorm'][i][:length]).reshape(length, 1))

    X_Train = np.array(X_Train)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    X_Train_All = np.concatenate([X_Train_All, X_Train], axis=0)
    Y_Train_BP_All = np.concatenate([Y_Train_BP_All, Y_Train_BP], axis=0)
    Y_Train_ABP_All = np.concatenate([Y_Train_ABP_All, Y_Train_ABP], axis=0)
    
    X_Train = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_3['PPG'].shape[0]), desc='Preparing Train Data  Part 3'):
        X_Train.append(np.array(fl_Train_3['PPG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_3['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_3['ABP_RNorm'][i][:length]).reshape(length, 1))

    X_Train = np.array(X_Train)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    X_Train_All = np.concatenate([X_Train_All, X_Train], axis=0)
    Y_Train_BP_All = np.concatenate([Y_Train_BP_All, Y_Train_BP], axis=0)
    Y_Train_ABP_All = np.concatenate([Y_Train_ABP_All, Y_Train_ABP], axis=0)
    
    X_Train_All = np.array(X_Train_All)
    Y_Train_BP_All = np.array(Y_Train_BP_All)
    Y_Train_ABP_All = np.array(Y_Train_ABP_All)
    print(X_Train_All.shape)
    print(Y_Train_BP_All.shape)
    print(Y_Train_ABP_All.shape)

elif num_channel == 2:
    PPG = []
    ECG = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_1['PPG'].shape[0]), desc='Preparing Train Data Part 1'):
        PPG.append(np.array(fl_Train_1['PPG'][i][:length]).reshape(length, 1))
        ECG.append(np.array(fl_Train_1['ECG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_1['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_1['ABP_RNorm'][i][:length]).reshape(length, 1))

    PPG = np.array(PPG)
    ECG = np.array(ECG)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    PPG_All = PPG
    ECG_All = ECG
    Y_Train_BP_All = Y_Train_BP
    Y_Train_ABP_All = Y_Train_ABP

    PPG = []
    ECG = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_2['PPG'].shape[0]), desc='Preparing Train Data Part 2'):
        PPG.append(np.array(fl_Train_2['PPG'][i][:length]).reshape(length, 1))
        ECG.append(np.array(fl_Train_2['ECG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_2['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_2['ABP_RNorm'][i][:length]).reshape(length, 1))

    PPG = np.array(PPG)
    ECG = np.array(ECG)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    PPG_All = np.concatenate([PPG_All, PPG], axis=0)
    ECG_All = np.concatenate([ECG_All, ECG], axis=0)
    Y_Train_BP_All = np.concatenate([Y_Train_BP_All, Y_Train_BP], axis=0)
    Y_Train_ABP_All = np.concatenate([Y_Train_ABP_All, Y_Train_ABP], axis=0)

    PPG = []
    ECG = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_3['PPG'].shape[0]), desc='Preparing Train Data Part 3'):
        PPG.append(np.array(fl_Train_3['PPG'][i][:length]).reshape(length, 1))
        ECG.append(np.array(fl_Train_3['ECG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_3['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_3['ABP_RNorm'][i][:length]).reshape(length, 1))

    PPG = np.array(PPG)
    ECG = np.array(ECG)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    PPG_All = np.concatenate([PPG_All, PPG], axis=0)
    ECG_All = np.concatenate([ECG_All, ECG], axis=0)
    Y_Train_BP_All = np.concatenate([Y_Train_BP_All, Y_Train_BP], axis=0)
    Y_Train_ABP_All = np.concatenate([Y_Train_ABP_All, Y_Train_ABP], axis=0)

    PPG_All = np.array(PPG_All)
    ECG_All = np.array(ECG_All)
    X_Train_All = np.squeeze(np.stack((PPG_All, ECG_All), axis=2))
    Y_Train_BP_All = np.array(Y_Train_BP_All)
    Y_Train_ABP_All = np.array(Y_Train_ABP_All)
    print(X_Train_All.shape)
    print(Y_Train_BP_All.shape)
    print(Y_Train_ABP_All.shape)

elif num_channel == 3:
    PPG = []
    VPG = []
    APG = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_1['PPG'].shape[0]), desc='Preparing Train Data Part 1'):
        PPG.append(np.array(fl_Train_1['PPG'][i][:length]).reshape(length, 1))
        VPG.append(np.array(fl_Train_1['VPG'][i][:length]).reshape(length, 1))
        APG.append(np.array(fl_Train_1['APG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_1['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_1['ABP_RNorm'][i][:length]).reshape(length, 1))

    PPG = np.array(PPG)
    VPG = np.array(VPG)
    APG = np.array(APG)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    PPG_All = PPG
    VPG_All = VPG
    APG_All = APG
    Y_Train_BP_All = Y_Train_BP
    Y_Train_ABP_All = Y_Train_ABP

    PPG = []
    VPG = []
    APG = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_2['PPG'].shape[0]), desc='Preparing Train Data Part 2'):
        PPG.append(np.array(fl_Train_2['PPG'][i][:length]).reshape(length, 1))
        VPG.append(np.array(fl_Train_2['VPG'][i][:length]).reshape(length, 1))
        APG.append(np.array(fl_Train_2['APG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_2['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_2['ABP_RNorm'][i][:length]).reshape(length, 1))

    PPG = np.array(PPG)
    VPG = np.array(VPG)
    APG = np.array(APG)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    PPG_All = np.concatenate([PPG_All, PPG], axis=0)
    VPG_All = np.concatenate([VPG_All, VPG], axis=0)
    APG_All = np.concatenate([APG_All, APG], axis=0)
    Y_Train_BP_All = np.concatenate([Y_Train_BP_All, Y_Train_BP], axis=0)
    Y_Train_ABP_All = np.concatenate([Y_Train_ABP_All, Y_Train_ABP], axis=0)

    PPG = []
    VPG = []
    APG = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_3['PPG'].shape[0]), desc='Preparing Train Data Part 3'):
        PPG.append(np.array(fl_Train_3['PPG'][i][:length]).reshape(length, 1))
        VPG.append(np.array(fl_Train_3['VPG'][i][:length]).reshape(length, 1))
        APG.append(np.array(fl_Train_3['APG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_3['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_3['ABP_RNorm'][i][:length]).reshape(length, 1))

    PPG = np.array(PPG)
    VPG = np.array(VPG)
    APG = np.array(APG)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    PPG_All = np.concatenate([PPG_All, PPG], axis=0)
    VPG_All = np.concatenate([VPG_All, VPG], axis=0)
    APG_All = np.concatenate([APG_All, APG], axis=0)
    Y_Train_BP_All = np.concatenate([Y_Train_BP_All, Y_Train_BP], axis=0)
    Y_Train_ABP_All = np.concatenate([Y_Train_ABP_All, Y_Train_ABP], axis=0)

    PPG_All = np.array(PPG_All)
    VPG_All = np.array(VPG_All)
    APG_All = np.array(APG_All)
    X_Train_All = np.squeeze(np.stack((PPG_All, VPG_All, APG_All), axis=2))
    Y_Train_BP_All = np.array(Y_Train_BP_All)
    Y_Train_ABP_All = np.array(Y_Train_ABP_All)
    print(X_Train_All.shape)
    print(Y_Train_BP_All.shape)
    print(Y_Train_ABP_All.shape)

elif num_channel == 4:
    PPG = []
    VPG = []
    APG = []
    ECG = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_1['PPG'].shape[0]), desc='Preparing Train Data Part 1'):
        PPG.append(np.array(fl_Train_1['PPG'][i][:length]).reshape(length, 1))
        VPG.append(np.array(fl_Train_1['VPG'][i][:length]).reshape(length, 1))
        APG.append(np.array(fl_Train_1['APG'][i][:length]).reshape(length, 1))
        ECG.append(np.array(fl_Train_1['ECG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_1['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_1['ABP_RNorm'][i][:length]).reshape(length, 1))

    PPG = np.array(PPG)
    VPG = np.array(VPG)
    APG = np.array(APG)
    ECG = np.array(ECG)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    PPG_All = PPG
    VPG_All = VPG
    APG_All = APG
    ECG_All = ECG
    Y_Train_BP_All = Y_Train_BP
    Y_Train_ABP_All = Y_Train_ABP

    PPG = []
    VPG = []
    APG = []
    ECG = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_2['PPG'].shape[0]), desc='Preparing Train Data Part 2'):
        PPG.append(np.array(fl_Train_2['PPG'][i][:length]).reshape(length, 1))
        VPG.append(np.array(fl_Train_2['VPG'][i][:length]).reshape(length, 1))
        APG.append(np.array(fl_Train_2['APG'][i][:length]).reshape(length, 1))
        ECG.append(np.array(fl_Train_2['ECG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_2['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_2['ABP_RNorm'][i][:length]).reshape(length, 1))

    PPG = np.array(PPG)
    VPG = np.array(VPG)
    APG = np.array(APG)
    ECG = np.array(ECG)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    PPG_All = np.concatenate([PPG_All, PPG], axis=0)
    VPG_All = np.concatenate([VPG_All, VPG], axis=0)
    APG_All = np.concatenate([APG_All, APG], axis=0)
    ECG_All = np.concatenate([ECG_All, ECG], axis=0)
    Y_Train_BP_All = np.concatenate([Y_Train_BP_All, Y_Train_BP], axis=0)
    Y_Train_ABP_All = np.concatenate([Y_Train_ABP_All, Y_Train_ABP], axis=0)

    PPG = []
    VPG = []
    APG = []
    ECG = []
    Y_Train_BP = []
    Y_Train_ABP = []
    for i in tqdm(range(0, fl_Train_3['PPG'].shape[0]), desc='Preparing Train Data Part 3'):
        PPG.append(np.array(fl_Train_3['PPG'][i][:length]).reshape(length, 1))
        VPG.append(np.array(fl_Train_3['VPG'][i][:length]).reshape(length, 1))
        APG.append(np.array(fl_Train_3['APG'][i][:length]).reshape(length, 1))
        ECG.append(np.array(fl_Train_3['ECG'][i][:length]).reshape(length, 1))
        Y_Train_BP.append(np.array(fl_Train_3['ABP'][i][:length]).reshape(length, 1))
        Y_Train_ABP.append(np.array(fl_Train_3['ABP_RNorm'][i][:length]).reshape(length, 1))

    PPG = np.array(PPG)
    VPG = np.array(VPG)
    APG = np.array(APG)
    ECG = np.array(ECG)
    Y_Train_BP = np.array(Y_Train_BP)
    Y_Train_ABP = np.array(Y_Train_ABP)
    PPG_All = np.concatenate([PPG_All, PPG], axis=0)
    VPG_All = np.concatenate([VPG_All, VPG], axis=0)
    APG_All = np.concatenate([APG_All, APG], axis=0)
    ECG_All = np.concatenate([ECG_All, ECG], axis=0)
    Y_Train_BP_All = np.concatenate([Y_Train_BP_All, Y_Train_BP], axis=0)
    Y_Train_ABP_All = np.concatenate([Y_Train_ABP_All, Y_Train_ABP], axis=0)

    PPG_All = np.array(PPG_All)
    VPG_All = np.array(VPG_All)
    APG_All = np.array(APG_All)
    ECG_All = np.array(ECG_All)
    X_Train_All = np.squeeze(np.stack((PPG_All, VPG_All, APG_All, ECG_All), axis=2))
    Y_Train_BP_All = np.array(Y_Train_BP_All)
    Y_Train_ABP_All = np.array(Y_Train_ABP_All)
    #
    print(X_Train_All.shape)
    print(Y_Train_BP_All.shape)
    print(Y_Train_ABP_All.shape)

Preparing Train Data Part 3: 100%|██████████████████████████████████████████████| 40110/40110 [03:53<00:00, 171.45it/s]


(140552, 1024, 4)
(140552, 1024, 1)
(140552, 1024, 1)


### Prepare Test Data

In [9]:
# intialize test data

length = 1024
X_Test = []
Y_Test_BP = []
Y_Test_ABP = []

if num_channel == 1:

    for i in tqdm(range(0, fl_Test['PPG'].shape[0]), desc='Preparing Test Data'):
        X_Test.append(np.array(fl_Test['PPG'][i][:length]).reshape(length, 1))    # ppg signal
        Y_Test_BP.append(np.array(fl_Test['ABP'][i][:length]).reshape(length, 1)) # abp signal
        Y_Test_ABP.append(np.array(fl_Test['ABP_RNorm'][i][:length]).reshape(length, 1))


    X_Test = np.array(X_Test)
    Y_Test_BP = np.array(Y_Test_BP)
    Y_Test_ABP = np.array(Y_Test_ABP)
    Y_Test_ABP.shape

elif num_channel == 2:
    PPG = []
    ECG = []

    for i in tqdm(range(0, fl_Test['PPG'].shape[0]), desc='Preparing Test Data'):
        PPG.append(np.array(fl_Test['PPG'][i][:length]).reshape(length, 1))       # ppg signal
        ECG.append(np.array(fl_Test['ECG'][i][:length]).reshape(length, 1))       # ecg signal
        Y_Test_BP.append(np.array(fl_Test['ABP'][i][:length]).reshape(length, 1)) # abp signal
        Y_Test_ABP.append(np.array(fl_Test['ABP_RNorm'][i][:length]).reshape(length, 1))


    PPG = np.array(PPG)
    ECG = np.array(ECG)
    X_Test = np.stack((PPG, ECG), axis=2)
    Y_Test_BP = np.array(Y_Test_BP)
    Y_Test_ABP = np.array(Y_Test_ABP)
    Y_Test_ABP.shape

elif num_channel == 3:
    PPG = []
    VPG = []
    APG = []

    for i in tqdm(range(0, fl_Test['PPG'].shape[0]), desc='Preparing Test Data'):
        PPG.append(np.array(fl_Test['PPG'][i][:length]).reshape(length, 1))       # ppg signal
        VPG.append(np.array(fl_Test['VPG'][i][:length]).reshape(length, 1))       # vpg signal
        APG.append(np.array(fl_Test['APG'][i][:length]).reshape(length, 1))       # apg signal
        Y_Test_BP.append(np.array(fl_Test['ABP'][i][:length]).reshape(length, 1)) # abp signal
        Y_Test_ABP.append(np.array(fl_Test['ABP_RNorm'][i][:length]).reshape(length, 1))


    PPG = np.array(PPG)
    VPG = np.array(VPG)
    APG = np.array(APG)
    X_Test = np.stack((PPG, VPG, APG), axis=2)
    Y_Test_BP = np.array(Y_Test_BP)
    Y_Test_ABP = np.array(Y_Test_ABP)
    Y_Test_ABP.shape

elif num_channel == 4:
    PPG = []
    VPG = []
    APG = []
    ECG = []

    for i in tqdm(range(0, fl_Test['PPG'].shape[0]), desc='Preparing Test Data'):
        PPG.append(np.array(fl_Test['PPG'][i][:length]).reshape(length, 1))       # ppg signal
        VPG.append(np.array(fl_Test['VPG'][i][:length]).reshape(length, 1))       # vpg signal
        APG.append(np.array(fl_Test['APG'][i][:length]).reshape(length, 1))       # apg signal
        ECG.append(np.array(fl_Test['ECG'][i][:length]).reshape(length, 1))       # apg signal
        Y_Test_BP.append(np.array(fl_Test['ABP'][i][:length]).reshape(length, 1)) # abp signal
        Y_Test_ABP.append(np.array(fl_Test['ABP_RNorm'][i][:length]).reshape(length, 1))


    PPG = np.array(PPG)
    VPG = np.array(VPG)
    APG = np.array(APG)
    ECG = np.array(ECG)
    X_Test = np.stack((PPG, VPG, APG, ECG), axis=2)
    Y_Test_BP = np.array(Y_Test_BP)
    Y_Test_ABP = np.array(Y_Test_ABP)
    #
    print(X_Test.shape)
    print(Y_Test_BP.shape)
    print(Y_Test_ABP.shape)

Preparing Test Data: 100%|██████████████████████████████████████████████████████| 50646/50646 [04:19<00:00, 194.96it/s]


(50646, 1024, 4, 1)
(50646, 1024, 1)
(50646, 1024, 1)


### Extract SBP and DBP Label Data from the Training Set

In [10]:
SBP_Train_1 = []
SBP_Train_2 = []
SBP_Train_3 = []
DBP_Train_1 = []
DBP_Train_2 = []
DBP_Train_3 = []

# SBP
for i in tqdm(range(0, fl_Train_1['SBP'].shape[0]), desc='Extracting Ground_Truth SBP from Train Data Part 1'):
    SBP_Train_1.append(np.array(fl_Train_1['SBP'][i][0]).reshape(1, 1)) 
SBP_Train_1 = np.squeeze(np.array(SBP_Train_1), axis=2)

for i in tqdm(range(0, fl_Train_2['SBP'].shape[0]), desc='Extracting Ground_Truth SBP from Train Data Part 2'):
    SBP_Train_2.append(np.array(fl_Train_2['SBP'][i][0]).reshape(1, 1)) 
SBP_Train_2 = np.squeeze(np.array(SBP_Train_2), axis=2)

for i in tqdm(range(0, fl_Train_3['SBP'].shape[0]), desc='Extracting Ground_Truth SBP from Train Data Part 3'):
    SBP_Train_3.append(np.array(fl_Train_3['SBP'][i][0]).reshape(1, 1)) 
SBP_Train_3 = np.squeeze(np.array(SBP_Train_3), axis=2)

SBP_Train = np.squeeze(np.concatenate((SBP_Train_1, SBP_Train_2, SBP_Train_3), axis=0))
print(SBP_Train.shape)

# DBP
for i in tqdm(range(0, fl_Train_1['DBP'].shape[0]), desc='Extracting Ground_Truth DBP from Train Data Part 1'):
    DBP_Train_1.append(np.array(fl_Train_1['DBP'][i][0]).reshape(1, 1)) 
DBP_Train_1 = np.squeeze(np.array(DBP_Train_1), axis=2)

for i in tqdm(range(0, fl_Train_2['DBP'].shape[0]), desc='Extracting Ground_Truth DBP from Train Data Part 2'):
    DBP_Train_2.append(np.array(fl_Train_2['DBP'][i][0]).reshape(1, 1)) 
DBP_Train_2 = np.squeeze(np.array(DBP_Train_2), axis=2)

for i in tqdm(range(0, fl_Train_3['DBP'].shape[0]), desc='Extracting Ground_Truth DBP from Train Data Part 3'):
    DBP_Train_3.append(np.array(fl_Train_3['DBP'][i][0]).reshape(1, 1)) 
DBP_Train_3 = np.squeeze(np.array(DBP_Train_3), axis=2)

DBP_Train = np.squeeze(np.concatenate((DBP_Train_1, DBP_Train_2, DBP_Train_3), axis=0))
#
print(DBP_Train.shape)

Extracting Ground_Truth SBP from Train Data Part 1: 100%|██████████████████████| 47441/47441 [00:19<00:00, 2445.77it/s]
Extracting Ground_Truth SBP from Train Data Part 2: 100%|██████████████████████| 53001/53001 [00:21<00:00, 2511.25it/s]
Extracting Ground_Truth SBP from Train Data Part 3: 100%|██████████████████████| 40110/40110 [00:16<00:00, 2390.65it/s]


(140552,)


Extracting Ground_Truth DBP from Train Data Part 1: 100%|██████████████████████| 47441/47441 [00:21<00:00, 2239.73it/s]
Extracting Ground_Truth DBP from Train Data Part 2: 100%|██████████████████████| 53001/53001 [00:21<00:00, 2454.30it/s]
Extracting Ground_Truth DBP from Train Data Part 3: 100%|██████████████████████| 40110/40110 [00:18<00:00, 2201.34it/s]


(140552,)


### Extract Ground Truth SBP and DBP from the Testing Set

In [11]:
SBP_Test = []
DBP_Test = []

# SBP
for i in tqdm(range(0, fl_Test['SBP'].shape[0]), desc='Extracting Ground_Truth SBP from Testing Set'):
    SBP_Test.append(np.array(fl_Test['SBP'][i][0]).reshape(1, 1)) 

SBP_Test = np.squeeze(np.array(SBP_Test),axis=2)

# DBP
for i in tqdm(range(0, fl_Test['DBP'].shape[0]), desc='Extracting Ground_Truth DBP from Testing Set'):
    DBP_Test.append(np.array(fl_Test['DBP'][i][0]).reshape(1, 1)) 

DBP_Test = np.squeeze(np.array(DBP_Test),axis=2)
#
print(SBP_Test.shape)
print(DBP_Test.shape)

Extracting Ground_Truth DBP from Testing Set: 100%|████████████████████████████| 50646/50646 [00:18<00:00, 2778.60it/s]

(50646, 1)
(50646, 1)


### Prepare ABP Ground Truth for PPG2ABP

In [12]:
ABP_GRND = []

for i in tqdm(range(0, fl_Test['ABP_GRND'].shape[0]), desc='Extracting Ground_Truth'):
    ABP_GRND.append(np.array(fl_Test['ABP_GRND'][i][:length]).reshape(length, 1)) 

ABP_GRND = np.array(ABP_GRND)
ABP_GRND.shape

Extracting Ground_Truth: 100%|█████████████████████████████████████████████████| 50646/50646 [00:26<00:00, 1915.31it/s]


(50646, 1024, 1)

### Train-Val Split [Test Data is Completely Independent]

In [13]:
X_Train1, X_Val1, Y_Train1, Y_Val1, Y_Train2, Y_Val2, SBP_Train1, SBP_Val1, DBP_Train1, DBP_Val1 = train_test_split(X_Train_All, Y_Train_BP_All, Y_Train_ABP_All, SBP_Train, DBP_Train, test_size=0.2, random_state=42)

MemoryError: Unable to allocate 878. MiB for an array with shape (112441, 1024, 1) and data type float64

In [ ]:
X_Test1 = X_Test
Y_Test1 = Y_Test_BP
Y_Test2 = Y_Test_ABP

### Save Splitted UCI Dataset

In [ ]:
if num_channel == 1:
  hf = h5py.File('UCI_Splitted_Data_1CH.h5', 'w')

elif num_channel == 2:
  hf = h5py.File('UCI_Splitted_Data_2CH.h5', 'w')

elif num_channel == 3:
  hf = h5py.File('UCI_Splitted_Data_3CH.h5', 'w')

elif num_channel == 4:
  hf = h5py.File('UCI_Splitted_Data_4CH.h5', 'w')
#
hf.create_dataset('X_Train1', data=X_Train1)
hf.create_dataset('X_Val1', data=X_Val1)
hf.create_dataset('Y_Train1', data=Y_Train1)
hf.create_dataset('Y_Val1', data=Y_Val1)
hf.create_dataset('Y_Train2', data=Y_Train2)
hf.create_dataset('Y_Val2', data=Y_Val2)
hf.create_dataset('X_Test1', data=X_Test1)
hf.create_dataset('Y_Test1', data=Y_Test1)
hf.create_dataset('Y_Test2', data=Y_Test2)
hf.create_dataset('SBP_Train1', data=SBP_Train1)
hf.create_dataset('SBP_Val1', data=SBP_Val1)
hf.create_dataset('DBP_Train1', data=DBP_Train1)
hf.create_dataset('DBP_Val1', data=DBP_Val1)
hf.create_dataset('SBP_Test', data=SBP_Test)
hf.create_dataset('DBP_Test', data=DBP_Test)
hf.create_dataset('ABP_GRND', data=ABP_GRND)
#
hf.close()

### Import Splitted UCI Dataset

In [ ]:
if num_channel == 1:
  hf = h5py.File('UCI_Splitted_Data_1CH.h5', 'r')
  hf.keys()

elif num_channel == 2:
  hf = h5py.File('UCI_Splitted_Data_2CH.h5', 'r')
  hf.keys()

elif num_channel == 3:
  hf = h5py.File('UCI_Splitted_Data_3CH.h5', 'r')
  hf.keys()

elif num_channel == 4:
  hf = h5py.File('UCI_Splitted_Data_4CH.h5', 'r')
  hf.keys()

In [ ]:
X_Train1 = np.array(hf.get('X_Train1'))
X_Val1 = np.array(hf.get('X_Val1'))
Y_Train1 = np.array(hf.get('Y_Train1'))
Y_Val1 = np.array(hf.get('Y_Val1'))
Y_Train2 = np.array(hf.get('Y_Train2'))
Y_Val2 = np.array(hf.get('Y_Val2'))
SBP_Train1 = np.array(hf.get('SBP_Train1'))
SBP_Val1 = np.array(hf.get('SBP_Val1'))
DBP_Train1 = np.array(hf.get('DBP_Train1'))
DBP_Val1 = np.array(hf.get('DBP_Val1'))
SBP_Test = np.array(hf.get('SBP_Test'))
DBP_Test = np.array(hf.get('DBP_Test'))
ABP_GRND = np.array(hf.get('ABP_GRND'))
X_Test1 = np.array(hf.get('X_Test1'))
Y_Test1 = np.array(hf.get('Y_Test1'))
Y_Test2 = np.array(hf.get('Y_Test2'))

### Garbage Collector

In [ ]:
import gc #Garbage Collector
PPG = None
VPG = None
APG = None
ECG = None
PPG_All = None
VPG_All = None
APG_All = None
ECG_All = None
X_Train_All = None
Y_Train_BP_All = None
Y_Train_ABP_All = None
fl_Train = None
fl_Test = None
X_Test = None
SBP_Train_1 = None
SBP_Train_2 = None
SBP_Train_3 = None
DBP_Train_1 = None
DBP_Train_2 = None
DBP_Train_3 = None
SBP_Train = None
DBP_Train = None
gc.collect()

# ABP Estimation

Configurations

In [ ]:
# Configurations
signal_length = 1024  # Length of each Segment
model_depth = 5  # Number of Level in the CNN Model
model_width = 64  # Width of the Initial Layer, subsequent layers start from here
kernel_size = 3  # Size of the Kernels/Filter
num_channel = 4  # Number of Channels in the Model
D_S = 0  # Turn on Deep Supervision
A_E = 0  # Turn on AutoEncoder Mode for Feature Extraction
A_G = 0  # Turn on for Guided Attention
LSTM = 0
num_dense_loop = 1
problem_type = 'Regression'
output_nums = 1  # Number of Class for Classification Problems, always '1' for Regression Problems
feature_number = 1024  # Number of Features to be Extracted, only required if the AutoEncoder Mode is turned on
model_name = 'NABNet'  # Dense_Inception_UNet

## Prepare Data for Deep Supervision

In [ ]:
def prepareTrainDict(y, model_depth, signal_length, model_name):
  def approximate(inp, w_len, signal_length):
    op = np.zeros((len(inp),signal_length//w_len))
    for i in range(0,signal_length,w_len):
      try:
        op[:,i//w_len] = np.mean(inp[:,i:i+w_len],axis=1)
      except Exception as e:
        print(e)
        print(i)
  	
    return op

  out = {}
  Y_Train_dict = {}
  out['out'] = np.array(y)
  Y_Train_dict['out'] = out['out']
  for i in range(1, (model_depth+1)):
    name = f'level{i}'
    if (model_name == 'UNet'):
      out[name] = np.expand_dims(approximate(np.squeeze(y), 2**i, signal_length),axis = 2)
    elif (model_name == 'UNetPP'):
      out[name] = np.expand_dims(approximate(np.squeeze(y), 2**0, signal_length),axis = 2)
    Y_Train_dict[f'level{i}'] = out[f'level{i}']
  
  return out, Y_Train_dict

In [ ]:
model_name_DS = 'UNetPP' # UNet or UNetPP (Two Types)
X_Train3 = X_Train1
X_Val3 = X_Val1
[Y_Train3, Y_Train_dict] = prepareTrainDict(Y_Train2, model_depth, signal_length, model_name_DS)
[Y_Val3, Y_Val_dict] = prepareTrainDict(Y_Val2, model_depth, signal_length, model_name_DS)

In [ ]:
loss_weights = np.zeros(model_depth)

for i in range(0, model_depth):
   loss_weights[i] = 1-(i*0.1)
   
loss_weights

In [ ]:
import gc #Garbage Collector
X_Test1 = None
X_Train1 = None
X_Val1 = None
Y_Train2 = None
Y_Test2 = None
Y_Val2 = None
Y_Train3 = None
Y_Val3 = None
PAVE2ABP_Network = None
gc.collect()

## Build Segmentation Model and Train

In [ ]:
# Build model for PPG2ABP Segmentation - Deep UNet Architecture
PAVE2ABP_Network = UNet(signal_length, model_depth, num_channel, model_width, kernel_size, problem_type=problem_type, output_nums=output_nums,
                    ds=D_S, ae=A_E, ag=A_G, is_transconv=True).UNetPP()
if D_S == 0:
    PAVE2ABP_Network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss=tf.keras.losses.MeanSquaredError(), metrics=tf.keras.metrics.MeanSquaredError())
elif D_S == 1:
    PAVE2ABP_Network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss=tf.keras.losses.MeanSquaredError(), metrics=tf.keras.metrics.MeanSquaredError(), loss_weights= loss_weights)

Load Previously Trained Weights to a Blank Model

In [ ]:
trained_model_path = 'Trained_Models/PAVE2ABP_UCI_'+model_name+'_'+str(model_depth)+'_'+str(model_width)+'_'+str(kernel_size)+'_'+str(num_channel)+'_Fold_'+str(fold_num)+'.h5'
PAVE2ABP_Network.load_weights(trained_model_path)

Or, Load Previously Trained Model

In [ ]:
trained_model_path = 'Trained_Models/PAVE2ABP_UCI_'+model_name+'_'+str(model_depth)+'_'+str(model_width)+'_'+str(kernel_size)+'_'+str(num_channel)+'_Fold_'+str(fold_num)+'.h5'
PAVE2ABP_Network = tf.keras.models.load_model(trained_model_path)
if D_S == 0:
  PAVE2ABP_Network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss=tf.keras.losses.MeanSquaredError(), metrics=tf.keras.metrics.MeanSquaredError())
elif D_S == 1:
  PAVE2ABP_Network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss=tf.keras.losses.MeanSquaredError(), metrics=tf.keras.metrics.MeanSquaredError(), loss_weights=loss_weights)

Compile and Train

In [ ]:
if D_S == 0:
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, mode='min'), tf.keras.callbacks.ModelCheckpoint('Trained_Models/PAVE2ABP_UCI_'+model_name+'_'
    +str(model_depth)+'_'+str(model_width)+'_'+str(kernel_size)+'_'+str(num_channel)+'_Fold_'+str(fold_num)+'.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
    
    history = PAVE2ABP_Network.fit(X_Train1, Y_Train2, epochs=200, batch_size=32, verbose=1, validation_data= (X_Val1, Y_Val2), shuffle= True, callbacks= callbacks)
elif D_S == 1:
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_out_loss', patience=15, mode='min'), tf.keras.callbacks.ModelCheckpoint('Trained_Models/PAVE2ABP_UCI_'+model_name+'_'
    +str(model_depth)+'_'+str(model_width)+'_'+str(kernel_size)+'_'+str(num_channel)+'_Fold_'+str(fold_num)+'.h5', verbose=1, monitor='val_out_loss', save_best_only=True, mode='min')]
    history = PAVE2ABP_Network.fit(X_Train3, Y_Train_dict, epochs=200, batch_size=8, verbose=1, validation_data= (X_Val3, Y_Val_dict), shuffle= True, callbacks= callbacks)

Plot History Plots (e.g., Loss, Accuracy, etc.)

In [ ]:
def history_plot(history):
  # list all dictionaries in history
  print(history.history.keys())
  # summarize history for error
  plt.figure(figsize=(12,10))
  plt.subplot(2,1,1)
  plt.plot(history.history['out_mean_squared_error'])
  plt.plot(history.history['val_out_mean_squared_error'])
  plt.title('Model Error Performance')
  plt.ylabel('Error')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()
  # summarize history for loss
  plt.figure(figsize=(12,10))
  plt.subplot(2,1,2)
  plt.plot(history.history['out_loss'])
  plt.plot(history.history['val_out_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper right')
  plt.show()
#
history_plot(history)

Test

In [ ]:
if D_S == 0:
    ABP_App = PAVE2ABP_Network.predict(X_Test1, verbose=1)
    print(ABP_App.shape)
elif D_S == 1:
    X_Test3 = X_Test1
    ABP_App_MultiLevels = PAVE2ABP_Network.predict(X_Test3, verbose=1)
    ABP_App = ABP_App_MultiLevels[0]
    print(ABP_App.shape)

Construction Error before Denormalizing

In [ ]:
[ABP_GRND_NEW, App_Predict_NEW] = Construction_Error_ABP(Y_Test2, ABP_App)

Import BP Predictions

In [ ]:
infile = open('Trained_Models/Preds_SBP_Channel_' + str(num_channel) + '_Fold_' + str(fold_num) + '.p','rb')
Preds_SBP_PAVE2BP = pickle.load(infile)
infile.close()
#
infile = open('Trained_Models/Preds_DBP_Channel_' + str(num_channel) + '_Fold_' + str(fold_num) + '.p','rb')
Preds_DBP_PAVE2BP = pickle.load(infile)
infile.close()
#
SBP = Preds_SBP_PAVE2BP.ravel()
DBP = Preds_DBP_PAVE2BP.ravel()

In [ ]:
ABP_App_Pred = []

for i in tqdm(range(0, ABP_App.shape[0]), desc='Denormalizing ABP'):
    ABP_App[i] = (ABP_App[i] - min(ABP_App[i]))/(max(ABP_App[i])-min(ABP_App[i])) # Range Normalize [0 1]
    ABP_App_Pred.append((ABP_App[i]*(SBP[i] - DBP[i])) + DBP[i])

ABP_App_Pred = np.array(ABP_App_Pred)
print(ABP_App_Pred.shape)

Visualize Outcome

In [ ]:
i = random.randint(0,len(ABP_GRND))
MAE = np.mean(np.abs(ABP_App_Pred[i].ravel()-ABP_GRND[i].ravel()))
plt.figure(figsize=(15,12))
plt.subplot(3,1,1)
plt.plot(X_Test1[i,:,0].ravel(), label='PPG');
plt.title(f"PPG -- Sample Number {i}")
plt.legend();
plt.subplot(3,1,2)
plt.plot(Y_Test2[i].ravel(), label='ABP_GT_Norm');
plt.plot(ABP_App[i].ravel(), label='Pred_Norm');
plt.title(f"ABP Normalized -- Sample Number {i}")
plt.legend();
plt.subplot(3,1,3)
plt.plot(ABP_GRND[i],label='ABP_GT');
plt.plot(ABP_App_Pred[i].ravel(), label='Pred');
plt.title(f"ABP -- Sample Number {i} -- MAE = {MAE}");
plt.legend();

Save Predicted ABP Waveforms

In [ ]:
hf = h5py.File('Results/ABP_Estimated_Fold_1.h5', 'w')
hf.create_dataset('ABP_RNorm', data=ABP_App)
hf.create_dataset('ABP', data=ABP_App_Pred)
hf.close()

## Evaluate ABP Estimation Performance

Construction Error

In [ ]:
[ABP_GRND_NEW, App_Predict_NEW] = Construction_Error_ABP(ABP_GRND, ABP_App_Pred)

BHS Metric

In [ ]:
BHS_Metric_ABP(ABP_GRND_NEW, App_Predict_NEW)

AAMI Standard

In [ ]:
calcErrorAAMI_ABP(ABP_GRND_NEW, App_Predict_NEW)

Evaluate Blood Pressure Level Classification from Estimated ABP Waveforms

In [ ]:
evaluate_BP_Classification_ABP(ABP_GRND_NEW, App_Predict_NEW)

Regression Plots

In [ ]:
regression_plot_ABP(ABP_GRND_NEW, App_Predict_NEW)

Bland-Altman Plots

In [ ]:
bland_altman_plot_ABP(ABP_GRND_NEW, App_Predict_NEW)

# Infinite Loop to Keep the Tab Alive

In [ ]:
while True:
    pass